# Subset barcode runs
This Python Jupyter notebook is creates a copy of the barcode runs file that contains just specific samples.

First, import Python modules:

In [1]:
import os

import pandas as pd

import yaml

Read in the "master" barcode runs CSV file and the runs used to compute the escape scores for each sample:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)

barcode_runs_file = os.path.join('../', config['barcode_runs'])
print(f"Reading barcode runs from {barcode_runs_file}")
barcode_runs = pd.read_csv(barcode_runs_file)

escape_score_samples_file = os.path.join('../', config['escape_score_samples'])
print(f"Reading escape score samples from {escape_score_samples_file}")
escape_score_samples = pd.read_csv(escape_score_samples_file)

Reading barcode runs from ../data/barcode_runs.csv
Reading escape score samples from ../results/escape_scores/samples.csv


Now read in the samples to subset:

In [3]:
samples_to_subset_df = pd.read_csv('samples_to_subset.csv')
print('Here are the samples we will subset to:')
display(samples_to_subset_df)

samples_to_subset = samples_to_subset_df['sample'].tolist()
assert len(samples_to_subset) == len(set(samples_to_subset)), 'duplicate samples to subset'

Here are the samples we will subset to:


,sample
0,LY-CoV555_400
1,CB6_400
2,LY-CoV555+LY-CoV016_400


Now get all the escape-score samples of interest:

In [4]:
if not set(samples_to_subset).issubset(escape_score_samples['name']):
    raise ValueError(f"Not all samples to subset are in the escape score samples.")
    
samples_subset = (
    escape_score_samples
    .query('name in @samples_to_subset')
    .reset_index(drop=True)
    )

print(f"Here are the samples for which we are subsetting barcode runs:")
display(samples_subset)

Here are the samples for which we are subsetting barcode runs:


,name,library,antibody,concentration,concentration_units,date,pre_sample,post_sample,frac_escape,pre_cells_sorted,post_cells_sorted
0,CB6_400,lib1,CB6,400,ng_per_mL,200904,expt_24-33-none-0-reference,expt_27-CB6-400-escape,0.222,1907893.0,160000000.0
1,CB6_400,lib2,CB6,400,ng_per_mL,200904,expt_24-33-none-0-reference,expt_27-CB6-400-escape,0.225,927804.0,160000000.0
2,LY-CoV555_400,lib1,LY-CoV555,400,ng_per_mL,210115,expt_82-86-none-0-reference,expt_82-LY-CoV555-400-escape,0.161,1622745.0,NaN
3,LY-CoV555_400,lib2,LY-CoV555,400,ng_per_mL,210115,expt_82-86-none-0-reference,expt_82-LY-CoV555-400-escape,0.169,1675156.0,NaN
4,LY-CoV555+LY-CoV016_400,lib1,LY-CoV555+LY-CoV016,400,ng_per_mL,210115,expt_82-86-none-0-reference,expt_83-LY-CoV555+LY-CoV016-400-escape,0.124,1293736.0,NaN
5,LY-CoV555+LY-CoV016_400,lib2,LY-CoV555+LY-CoV016,400,ng_per_mL,210115,expt_82-86-none-0-reference,expt_83-LY-CoV555+LY-CoV016-400-escape,0.123,1265658.0,NaN


Now unfold the samples of interest into the actual relevant barcode runs (this requires getting both the pre- and post-selection run for each sample):

In [5]:
barcode_runs_subset = (
    samples_subset
    .melt(id_vars=['name', 'library'],
          value_vars=['pre_sample', 'post_sample'],
          value_name='sample',
          var_name='sample_type')
    [['library', 'sample']]
    .drop_duplicates()
    .reset_index(drop=True)
    .merge(barcode_runs,
           how='left',
           on=['library', 'sample'],
           validate='one_to_one',
           )
    )

assert barcode_runs_subset['R1'].notnull().all(), 'some barcode runs missing R1'

barcode_runs_subset_file = 'barcode_runs_subset.csv'
print(f"Here are the subsetted barcode runs. Writing to {barcode_runs_subset_file}")
barcode_runs_subset.to_csv(barcode_runs_subset_file, index=False)
display(barcode_runs_subset)

Here are the subsetted barcode runs. Writing to barcode_runs_subset.csv


,library,sample,date,experiment,antibody,concentration,concentration_units,group,selection,frac_escape,cells_sorted,R1
0,lib1,expt_24-33-none-0-reference,200904,expt_24-33,none,0,ng_per_mL,clinical_serum,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200914_D00300_10...
1,lib2,expt_24-33-none-0-reference,200904,expt_24-33,none,0,ng_per_mL,clinical_serum,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200914_D00300_10...
2,lib1,expt_82-86-none-0-reference,210115,expt_82-86,none,0,ng_per_mL,Baker+LY,reference,NaN,NaN,/shared/ngs/illumina/tstarr/210119_D00300_1157...
3,lib2,expt_82-86-none-0-reference,210115,expt_82-86,none,0,ng_per_mL,Baker+LY,reference,NaN,NaN,/shared/ngs/illumina/tstarr/210119_D00300_1157...
4,lib1,expt_27-CB6-400-escape,200904,expt_27,CB6,400,ng_per_mL,clinical_serum,escape,0.222,1907893.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
5,lib2,expt_27-CB6-400-escape,200904,expt_27,CB6,400,ng_per_mL,clinical_serum,escape,0.225,927804.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
6,lib1,expt_82-LY-CoV555-400-escape,210115,expt_82,LY-CoV555,400,ng_per_mL,LY,escape,0.161,1622745.0,/shared/ngs/illumina/tstarr/210119_D00300_1157...
7,lib2,expt_82-LY-CoV555-400-escape,210115,expt_82,LY-CoV555,400,ng_per_mL,LY,escape,0.169,1675156.0,/shared/ngs/illumina/tstarr/210119_D00300_1157...
8,lib1,expt_83-LY-CoV555+LY-CoV016-400-escape,210115,expt_83,LY-CoV555+LY-CoV016,400,ng_per_mL,LY,escape,0.124,1293736.0,/shared/ngs/illumina/tstarr/210119_D00300_1157...
9,lib2,expt_83-LY-CoV555+LY-CoV016-400-escape,210115,expt_83,LY-CoV555+LY-CoV016,400,ng_per_mL,LY,escape,0.123,1265658.0,/shared/ngs/illumina/tstarr/210119_D00300_1157...
